In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pymc3 as pm
import theano.tensor as tt


SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

SEED = 35010732 # from random.org
np.random.seed(SEED)

print(plt.style.available)
plt.style.use('seaborn-white')

The conditional probability for a OU process $p(x,t|x_{0},0)$ is

$$p(x,t|x_{0},0)=\frac{1}{\sqrt{2\pi A(1-B^{2})}}\exp \left(-\frac{(x-Bx_{0})^{2}}{2A(1-B^{2})}\right)$$

In [ ]:
class Ornstein_Uhlenbeck(pm.Continuous):
    """
    Ornstein-Uhlenbeck Process
    Parameters
    ----------
    B : tensor
        B > 0, B = exp(-(D/A)*delta_t)
    A : tensor
        A > 0, amplitude of fluctuation <x**2>=A
    delta_t: scalar
        delta_t > 0, time step
    """

    def __init__(self, A=None, B=None,
                 *args, **kwargs):
        super(Ornstein_Uhlenbeck, self).__init__(*args, **kwargs)
        self.A = A
        self.B = B
        self.mean = 0.

    def logp(self, x):
        A = self.A
        B = self.B

        x_im1 = x[:-1]
        x_i = x[1:]

        ou_like = pm.Normal.dist(mu=x_im1*B, tau=1.0/A/(1-B**2)).logp(x_i)
        return pm.Normal.dist(mu=0.0,tau=1.0/A).logp(x[0]) + tt.sum(ou_like)


In [ ]:
data = np.load("OUmt_sN05.npy")
data = data[:2]

In [ ]:
a_bound = 10
result_df = pd.DataFrame(columns=['dt','A', 'dA','B','dB','s','ds'])
for dataset in data:
    delta_t = dataset[0]
    ts = dataset[1:]
    print(delta_t)
    with pm.Model() as model:
        B = pm.Beta('B', alpha=1.0,beta=1.0)
        A = pm.Uniform('A', lower=0, upper=a_bound)
        sigma = pm.Uniform('sigma',lower=0,upper=5)

        path = Ornstein_Uhlenbeck('path',A=A, B=B,shape=len(ts))
        dataObs = pm.Normal('dataObs',mu=path,sigma=sigma,observed=ts)
        trace = pm.sample(2000,cores=4)
        
    a_mean = trace['A'].mean()
    b_mean = trace['B'].mean()
    a_std = trace['A'].std()
    b_std = trace['B'].std()
    sigma_mean = trace['sigma'].mean()
    sigma_std = trace['sigma'].std()
    
    result_df = result_df.append({'dt':delta_t,
                      'A':a_mean,
                      'dA':a_std,
                      'B':b_mean,
                      'dB':b_std,
                      's':sigma_mean,
                      'ds':sigma_std},ignore_index=True)


In [ ]:
tau = -delta_t_list/np.log(result_array.T[2])
dtau = delta_t_list*result_array.T[3]/result_array.T[2]/np.log(result_array.T[2])**2

In [ ]:
plt.plot(delta_t_list,result_array.T[6],"o")
plt.xlabel(r'$\Delta t/\tau$')
plt.ylabel(r'$\sigma_{GT-model}$')

In [ ]:
plt.errorbar(delta_t_list,result_array.T[0],yerr=result_array.T[1],fmt="o",label="A")
plt.errorbar(delta_t_list,tau,dtau,fmt="o",label=r'$\tau$')
plt.legend(loc="upper left")

In [ ]:
plt.errorbar(delta_t_list,result_array.T[4],yerr=result_array.T[5],fmt="o")
plt.xlabel(r'$\Delta t/\tau$')
plt.ylabel(r'$\sigma_{noise}$')